In [1]:
import torch, torchvision, random, numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import EMNIST
from torchvision import transforms, models
from tqdm.notebook import tqdm
from torchinfo import summary

In [2]:
SEED        = 42
BATCH_SIZE  = 256
EPOCHS      = 25
BASE_LR     = 1e-3
WEIGHT_DECAY= 5e-4
NUM_WORKERS = 2
AMP         = torch.cuda.is_available()

def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device, "AMP:", AMP)

Device: cuda AMP: True


In [3]:
mean = (0.1736,)
std  = (0.3317,)
train_tf = transforms.Compose([
    transforms.RandomCrop(28, padding=2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

data_root = "./data"
train_set = EMNIST(root=data_root, split="letters", train=True, download=True, transform=train_tf)
test_set  = EMNIST(root=data_root, split="letters", train=False, download=True, transform=test_tf)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f"Train: {len(train_set)}  Test: {len(test_set)}")

100%|██████████| 562M/562M [00:02<00:00, 203MB/s]


Train: 124800  Test: 20800


In [4]:
class ResNet18FC(nn.Module):
    def __init__(self, num_classes: int = 26):
        super().__init__()
        base = models.resnet18(weights=None)
        base.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        base.maxpool = nn.Identity()
        in_feats = base.fc.in_features
        base.fc = nn.Linear(in_feats, num_classes)  # standard FC classifier
        self.net = base

    def forward(self, x):
        return self.net(x)

model = ResNet18FC().to(device)
print("Params (baseline ResNet‑18):", sum(p.numel() for p in model.parameters())/1e6, "M")

Params (baseline ResNet‑18): 11.181018 M


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=AMP)

/tmp/ipykernel_19/4179177595.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=AMP)


In [6]:
@torch.no_grad()
def evaluate():
    model.eval(); corr=tot=0; loss_sum=0
    loop = tqdm(test_loader, leave=False)
    for imgs,targets in loop:
        imgs,targets = imgs.to(device), (targets-1).to(device)
        with torch.cuda.amp.autocast(enabled=AMP):
            out = model(imgs)
            loss = criterion(out,targets)
        loss_sum += loss.item()*imgs.size(0)
        corr += (out.argmax(1)==targets).sum().item(); tot += targets.size(0)
        loop.set_description("Eval"); loop.set_postfix(acc=100*corr/tot)
    return loss_sum/tot, corr/tot

def train_one_epoch(epoch):
    model.train(); run_loss=corr=tot=0
    loop = tqdm(train_loader, leave=False)
    for imgs,targets in loop:
        imgs,targets = imgs.to(device), (targets-1).to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=AMP):
            outputs = model(imgs)
            loss = criterion(outputs, targets)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        run_loss += loss.item()*imgs.size(0)
        corr += (outputs.argmax(1)==targets).sum().item(); tot += targets.size(0)
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=run_loss/tot, acc=100*corr/tot)

best_acc=0
for ep in range(1,EPOCHS+1):
    train_one_epoch(ep)
    val_loss,val_acc = evaluate()
    scheduler.step()
    print(f"Ep {ep}: val_acc={val_acc*100:.2f}%")
    if val_acc > best_acc:
        best_acc = val_acc; torch.save(model.state_dict(), "best_spinalresnet_emnist.pth")
        print("✓ Saved best model")

print("Best accuracy:", best_acc*100)


  0%|          | 0/488 [00:00<?, ?it/s]

/tmp/ipykernel_19/2135158058.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=AMP):


  0%|          | 0/82 [00:00<?, ?it/s]

/tmp/ipykernel_19/2135158058.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=AMP):


Ep 1: val_acc=92.32%
✓ Saved best model


...

  0%|          | 0/82 [00:00<?, ?it/s]

Ep 24: val_acc=95.77%
✓ Saved best model


  0%|          | 0/488 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

Ep 25: val_acc=95.75%
Best accuracy: 95.77403846153845
